In [19]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns

import harness

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)

In [4]:
add_features = harness.chain(
    harness.add_date_parts, harness.add_haul, harness.add_weather
)

In [5]:
flights_with_features = add_features(flights_clean)

In [6]:
x_transform = harness.chain(
    add_features,
    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric
)

In [7]:
transformer = harness.DataTransformer(
    x_transform, harness.normalize, harness.unnormalize
)

In [8]:
x_train, y_train = transformer.extract_transform(flights_clean)

In [13]:
def train_model():
    linreg = lm.Ridge(random_state=42)
    grid_search = ms.GridSearchCV(
        linreg, dict(alpha=[10 ** i for i in range(-6, 7)])
    )
    grid_search.fit(x_train.values, y_train.values)
    harness.save(grid_search.best_estimator_, 'everything_linear_model')

In [20]:
#train_model()

In [21]:
linreg = harness.load('everything_linear_model')
linreg_model = harness.TrainedModel(linreg, transformer)

In [23]:
linreg_model.validate(harness.clean_train(flights_test))

0.05961365182340628

In [ ]:
linreg_model.submit(
    'data/test.csv', 'time_of_day_submission.csv', 'predicted_delay'
)

So this model is pretty bad. It's worse than just assuming the average delay. We can see why looking at the graphs below. Time of day predicts taxi time well, but this doesn't translate well into actual delays measurements.

In [ ]:
sns.scatterplot(x=flights_clean.crs_dep_time, y=y_train.arr_delay, alpha=0.05)
plt.show()

In [ ]:
sns.scatterplot(x=flights_clean.crs_arr_time, y=y_train.arr_delay, alpha=0.05)
plt.show()